In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\myass\AppData\Local\Temp\ipykernel_19804\354183679.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Explore data

In [2]:
train_data = pd.read_csv("input/nlp-getting-started/train.csv")
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [4]:
train_data.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


* **Note** we note that keyword is not a real good classifier to obtain our target, and also we note that location can be an important feature to add with the text to predict our target.

# Preprocessing data

## Preprocessing location

In [5]:
featuretest_data=pd.read_pickle("location.pkl")
featuretest_data.sample(20)

,id,keyword,location,text,target,geolocation,geolocation_converted
106,156,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0,"(United States, (39.7837304, -100.445882))",True
6654,9537,terrorist,Florida,@lovemywife1983 @FoxNews like shedid on 9/11 4...,1,"(Florida, United States, (27.7567667, -81.4639...",True
2724,3913,devastated,-?s?s?j??s-,Obama Declares Disaster for Typhoon-Devastated...,1,NaN,False
7515,10748,wreckage,Africa,Malaysia PM confirms wreckage belongs to MH370...,1,"(Africa, (11.5024338, 17.7578122))",True
7110,10187,violent%20storm,Costa Rica,RT '@NASASolarSystem: Jupiter's Red Spot is a ...,0,"(Costa Rica, (10.2735633, -84.0739102))",True
961,1390,body%20bag,US,Photo: Bath &amp; Body Works cosmetic bag in p...,0,"(United States, (39.7837304, -100.445882))",True
2962,4255,drowning,"Chicago, IL",Family mourns drowning of 'superhero' toddler ...,0,"(Chicago, Cook County, Illinois, United States...",True
1398,2017,casualties,Canadian bread,@LibertarianLuke I'm all for that to be honest...,0,"(COBS Bread, 15941, 97 Street NW, Eaux Claires...",True
3871,5503,flames,Santo Domingo Alma Rosa,SoloQuiero Maryland mansion fire that killed 6...,1,"(Alma Rosa, Santo Domingo Este, Santo Domingo,...",True
2286,3281,demolish,"ATL, GA",@MarioMaraczi I'm watching it right now. He f...,0,(Hartsfield–Jackson Atlanta International Airp...,True


In [6]:
#Show detailed geolocation object from the first entry
featuretest_data.iloc[0]["geolocation"].raw

{'place_id': 243595575,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 162378,
 'lat': '52.4796992',
 'lon': '-1.9026911',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 16,
 'importance': 0.6734272656139734,
 'addresstype': 'city',
 'name': 'Birmingham',
 'display_name': 'Birmingham, West Midlands Combined Authority, England, United Kingdom',
 'address': {'city': 'Birmingham',
  'ISO3166-2-lvl8': 'GB-BIR',
  'state_district': 'West Midlands Combined Authority',
  'state': 'England',
  'ISO3166-2-lvl4': 'GB-ENG',
  'country': 'United Kingdom',
  'country_code': 'gb'},
 'boundingbox': ['52.3810530', '52.6087058', '-2.0336486', '-1.7288417']}

In [7]:
## Extract country from geolocation
from functools import partial

def _geolocation_to_scope(geolocation, scope:str):
    if geolocation is None or pd.isna(geolocation) or scope not in geolocation.raw["address"]:
        return np.nan
    return geolocation.raw["address"][scope]

def extract_from_geolocation(df:pd.DataFrame, scope:str):
    df[scope]=df["geolocation"].map(partial(_geolocation_to_scope,scope=scope))
    return df
extract_from_geolocation(featuretest_data,scope="country_code")

,id,keyword,location,text,target,geolocation,geolocation_converted,country_code
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,"(Birmingham, West Midlands Combined Authority,...",True,gb
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,NaN,False,NaN
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,"(Africa, (11.5024338, 17.7578122))",True,NaN
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,"(Philadelphia, Philadelphia County, Pennsylvan...",True,us
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"(London, Greater London, England, United Kingd...",True,gb
...,...,...,...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0,"(تونس, (33.8439408, 9.400138))",True,tn
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0,NaN,False,NaN
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,"(Vancouver, Metro Vancouver Regional District,...",True,ca
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,"(London, Greater London, England, United Kingd...",True,gb


## Missing Values


### Missing location

- Sample randomly from the country with weights based on occurences on the dataset

In [8]:
train_data_without_location = train_data[train_data["location"].isna()]
train_data_without_location

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
# Concatenate dataframe without location + preprocessing location (executed only on existing locations)
all_columns = set(featuretest_data.columns).union(train_data_without_location.columns)

# Concatenate dataframes and fill missing values with None
train_data_preprocessed_location = pd.concat([featuretest_data.reindex(columns=all_columns),
                             train_data_without_location.reindex(columns=all_columns)], ignore_index=True)
train_data_preprocessed_location

,country_code,target,id,geolocation,text,location,keyword,geolocation_converted
0,gb,1,48,"(Birmingham, West Midlands Combined Authority,...",@bbcmtd Wholesale Markets ablaze http://t.co/l...,Birmingham,ablaze,1.0
1,NaN,0,49,NaN,We always try to bring the heavy. #metal #RT h...,Est. September 2012 - Bristol,ablaze,0.0
2,NaN,1,50,"(Africa, (11.5024338, 17.7578122))",#AFRICANBAZE: Breaking news:Nigeria flag set a...,AFRICA,ablaze,1.0
3,us,0,52,"(Philadelphia, Philadelphia County, Pennsylvan...",Crying out for more! Set me ablaze,"Philadelphia, PA",ablaze,1.0
4,gb,0,53,"(London, Greater London, England, United Kingd...",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,"London, UK",ablaze,1.0
...,...,...,...,...,...,...,...,...
7608,NaN,1,10869,NaN,Two giant cranes holding a bridge collapse int...,NaN,NaN,NaN
7609,NaN,1,10870,NaN,@aria_ahrary @TheTawniest The out of control w...,NaN,NaN,NaN
7610,NaN,1,10871,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,NaN,NaN,NaN
7611,NaN,1,10872,NaN,Police investigating after an e-bike collided ...,NaN,NaN,NaN


In [10]:
missing_indices= train_data_preprocessed_location["country_code"].isna()
number_missing=missing_indices.sum()
class_counts = train_data_preprocessed_location["country_code"].value_counts(normalize=True)
random_classes = np.random.choice(class_counts.index, size=number_missing, p=class_counts.values)

# Fill missing values with randomly selected values
train_data_preprocessed_location.loc[missing_indices,"country_code"] = random_classes

In [11]:
train_data_preprocessed_location

,country_code,target,id,geolocation,text,location,keyword,geolocation_converted
0,gb,1,48,"(Birmingham, West Midlands Combined Authority,...",@bbcmtd Wholesale Markets ablaze http://t.co/l...,Birmingham,ablaze,1.0
1,us,0,49,NaN,We always try to bring the heavy. #metal #RT h...,Est. September 2012 - Bristol,ablaze,0.0
2,us,1,50,"(Africa, (11.5024338, 17.7578122))",#AFRICANBAZE: Breaking news:Nigeria flag set a...,AFRICA,ablaze,1.0
3,us,0,52,"(Philadelphia, Philadelphia County, Pennsylvan...",Crying out for more! Set me ablaze,"Philadelphia, PA",ablaze,1.0
4,gb,0,53,"(London, Greater London, England, United Kingd...",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,"London, UK",ablaze,1.0
...,...,...,...,...,...,...,...,...
7608,nz,1,10869,NaN,Two giant cranes holding a bridge collapse int...,NaN,NaN,NaN
7609,ch,1,10870,NaN,@aria_ahrary @TheTawniest The out of control w...,NaN,NaN,NaN
7610,us,1,10871,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,NaN,NaN,NaN
7611,in,1,10872,NaN,Police investigating after an e-bike collided ...,NaN,NaN,NaN


### Missing keywords

In [12]:
import pandas as pd
import importlib 
import preprocess.keywords as kw
importlib.reload(kw)
layer=kw.PreprocessKeywords()
layer.preprocess(train_data_preprocessed_location)


,country_code,target,id,geolocation,text,location,keyword,geolocation_converted,keyword_prep
0,gb,1,48,"(Birmingham, West Midlands Combined Authority,...",@bbcmtd Wholesale Markets ablaze http://t.co/l...,Birmingham,ablaze,1.0,ablaze
1,us,0,49,NaN,We always try to bring the heavy. #metal #RT h...,Est. September 2012 - Bristol,ablaze,0.0,ablaze
2,us,1,50,"(Africa, (11.5024338, 17.7578122))",#AFRICANBAZE: Breaking news:Nigeria flag set a...,AFRICA,ablaze,1.0,ablaze
3,us,0,52,"(Philadelphia, Philadelphia County, Pennsylvan...",Crying out for more! Set me ablaze,"Philadelphia, PA",ablaze,1.0,ablaze
4,gb,0,53,"(London, Greater London, England, United Kingd...",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,"London, UK",ablaze,1.0,ablaze
...,...,...,...,...,...,...,...,...,...
7608,nz,1,10869,NaN,Two giant cranes holding a bridge collapse int...,NaN,NaN,NaN,collapse
7609,ch,1,10870,NaN,@aria_ahrary @TheTawniest The out of control w...,NaN,NaN,NaN,@aria_ahrary
7610,us,1,10871,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,NaN,NaN,NaN,UTC]?5
7611,in,1,10872,NaN,Police investigating after an e-bike collided ...,NaN,NaN,NaN,collided


## Preprocessing text column
* Remove Http link in text
* Remove stopwords, punctuations
* Tolower

In [18]:
train_data_preprocessed_location.country_code.isnull().sum()

0

In [19]:
train_data_preprocessed_location.columns

Index(['country_code', 'target', 'id', 'geolocation', 'text', 'location',
       'keyword', 'geolocation_converted', 'keyword_prep'],
      dtype='object')

In [22]:
train_data_preprocessed_location_keyword = train_data_preprocessed_location[["country_code","keyword_prep","text","target"]].copy()
train_data_preprocessed_location_keyword

,country_code,keyword_prep,text,target
0,gb,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,us,ablaze,We always try to bring the heavy. #metal #RT h...,0
2,us,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,us,ablaze,Crying out for more! Set me ablaze,0
4,gb,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...
7608,nz,collapse,Two giant cranes holding a bridge collapse int...,1
7609,ch,@aria_ahrary,@aria_ahrary @TheTawniest The out of control w...,1
7610,us,UTC]?5,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,in,collided,Police investigating after an e-bike collided ...,1


In [40]:
from preprocess.keywords import load_nlp
nlp = load_nlp()
def remove_stop_words(text):
    text = text.strip()
    list_items = nlp(text)
    list_items_without_stops_words = [item.text for item in list_items if not item.is_stop]
    text_without_stopwords = " "
    return text_without_stopwords.join(list_items_without_stops_words)
text = train_data_preprocessed_location_keyword.iloc[1]["text"]
print(type(text))
print(text)
remove_stop_words(text)

<class 'str'>
We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw


'try bring heavy . # metal # RT http://t.co/YAo1e0xngw'

In [ ]:
# remove http
def remove_http(text):
    text = [word for word in text.split() if not word.startswith("http")]
    text_without_http =" "
    return text_without_http.join(text)


In [43]:
import re
REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = remove_stop_words(text)
    return text
text = train_data_preprocessed_location_keyword.iloc[1]["text"]
text = clean_text(text=text)
print(text)

try bring heavy # metal # rt http   tco yao1e0xngw
